# Scrittopoli - Giornata 2

In [1]:
%matplotlib inline

# Import the necessary packages and modules
import matplotlib
matplotlib.style.use('ggplot')
import matplotlib.pyplot as plt
import matplotlib.dates  as mpdt
import matplotlib.ticker as mptk
import matplotlib.mlab as mlab

import numpy as np
import pytz
mytz=pytz.timezone("Europe/Rome")
import pandas as pd
import datetime as dt
import math
import importlib
import random
import itertools
import csv
import string

import mylib.spreadsheet as ms
import mylib.elo as melo
import mylib.utility as mu


In [7]:
# Generale

day="20171214_142354" # definitivo
calendario_csv="data/calendario_%s.csv" % day
gironi_csv="data/gironi_%s.csv" % day
tabellone_csv="data/tabellone_%s.csv" % day
ranking_csv="data/ranking_%s.csv" % day
scrittopoli_csv="data/scrittopoli_%s.csv" % day
elo_giocatori_csv="data/elo_giocatori_%s.csv" % day
elo_squadre_csv="data/elo_squadre_%s.csv" % day

calendario=pd.read_csv(calendario_csv,sep=":",quotechar='"',header=0,index_col=1).sort_index()
calendario=calendario.set_index(["girone","partita"], append=True)

gironi=pd.read_csv(gironi_csv,sep=":",quotechar='"',header=0)

tabellone=pd.read_csv(tabellone_csv,sep=",",quotechar='"',header=0,index_col=0)
elo_giocatori=pd.read_csv(elo_giocatori_csv,sep=",",quotechar='"',header=0,index_col=0)

ranking=pd.read_csv(ranking_csv,sep=",",quotechar='"',header=0)
ranking=ranking[ranking["squadra"]!="(riposo)"]
ranking=ranking.set_index("squadra")

scrittopoli=pd.read_csv(scrittopoli_csv,sep=",",quotechar='"',header=0)
scrittopoli=scrittopoli.set_index(["squadra","giocatore"])

elo_squadre=pd.read_csv(elo_squadre_csv,sep=",",quotechar='"',header=0)
elo_squadre=elo_squadre.set_index(["squadra","giocatore"])

# Giornata 1

label="giornata_1"
schedina_csv="data/schedina_"+label+".csv"
classifica_csv="data/classifica_"+label+".csv"
elo_g_csv="data/elo_g_"+label+".csv"

schedina_g1=pd.read_csv(schedina_csv,sep=",",quotechar='"')
classifica_g1=pd.read_csv(classifica_csv,sep=",",quotechar='"').set_index(["girone","squadra"])
elo_g_g1=pd.read_csv(elo_g_csv,sep=",",quotechar='"').set_index("giocatore")

# Giornata 2

label="giornata_2"
schedina_csv="data/schedina_"+label+".csv"
classifica_csv="data/classifica_"+label+".csv"
elo_g_csv="data/elo_g_"+label+".csv"

#schedina_g2=pd.read_csv(schedina_csv,sep=",",quotechar='"')
#classifica_g2=pd.read_csv(classifica_csv,sep=",",quotechar='"').set_index(["girone","squadra"])
#elo_g_g2=pd.read_csv(elo_g_csv,sep=",",quotechar='"').set_index("giocatore")


GIORNATA=3

## Come si svolgono gli incontri

Ogni capitano presenta la formazione (due titolari e una riserva). I due titolari vengono accoppiati agli avversari sulla base del punteggio Elo (i due più alti tra loro e i due più bassi tra loro).

Inseriscono i due racconti. Cose da controllare:
* link al racconto;
* tag;
* lunghezza del racconto;
* link al commento;
* commento valido.

Per la votazione ogni squadra presenta l'elenco dei voti da dare a quelle degli altri gironi. Per ogni partita danno quattro punteggi. Oltre a queste ci sono le penalità dell'arbitro.


In [8]:
quadro=calendario.loc[GIORNATA].copy()

riposano=quadro[quadro["squadra 2"]=="(riposo)"][["squadra 1"]]
riposano




,,squadra 1
girone,partita,
A,1,D Euforiche


In [9]:
quadro=quadro[quadro["squadra 2"]!="(riposo)"]
quadro["vs"]="contro"

quadro=quadro[["squadra 1","vs","squadra 2"]]

quadro=quadro.sort_index()
quadro

squadra 1      vs                      squadra 2
girone partita                                                           
A      2              La threesome  contro                     Gli ultimi
B      1               Green House  contro                 the 3 BrOthers
       2                I Bananari  contro  Gli scrittori del quartierino
C      1             Living Stones  contro             Armata Brancaleone
       2        Retroguardia Trash  contro                    Le TreMende

In [10]:
importlib.reload(mu)
mu.to_table_html_multi(quadro)


<table style='margin:auto;' border='1' width='90%'>
<th>girone</th>
<th>partita</th>
<th>squadra 1</th>
<th>vs</th>
<th>squadra 2</th>
</tr>
<tr>
<th rowspan='1'>A</th>
<th rowspan='1'>2</th>
<td style='text-align:center;'>La threesome</td>
<td style='text-align:center;'>contro</td>
<td style='text-align:center;'>Gli ultimi</td>
</tr>
<tr>
<th rowspan='2'>B</th>
<th rowspan='1'>1</th>
<td style='text-align:center;'>Green House</td>
<td style='text-align:center;'>contro</td>
<td style='text-align:center;'>the 3 BrOthers</td>
</tr>
<tr>
<th rowspan='1'>2</th>
<td style='text-align:center;'>I Bananari</td>
<td style='text-align:center;'>contro</td>
<td style='text-align:center;'>Gli scrittori del quartierino</td>
</tr>
<tr>
<th rowspan='2'>C</th>
<th rowspan='1'>1</th>
<td style='text-align:center;'>Living Stones</td>
<td style='text-align:center;'>contro</td>
<td style='text-align:center;'>Armata Brancaleone</td>
</tr>
<tr>
<th rowspan='1'>2</th>
<td style='text-align:center;'>Retroguard

## Fase 1. Dalle formazioni ai match

In [14]:
squadre=scrittopoli.copy()
squadre=squadre.reset_index()[["squadra","giocatore"]].set_index("squadra")

importlib.reload(ms)
s=ms.N2017Spreadsheet()

#old_data=s.get_formazioni(GIORNATA)
old_data=None
s.set_formazioni(GIORNATA,riposano["squadra 1"].values,squadre.index.unique(),old_data)


#formazioni=s.get_formazioni(GIORNATA)
#formazioni



In [44]:
# Versione random di prova, da sostituire con lettura della formazione via spreadsheet sotto
#data=[]
#for sq in squadre.index.unique():
#    capitano=squadre.loc[sq].sample(n=1).iloc[0]["giocatore"]
#    riserva=squadre.loc[sq].sample(n=1).iloc[0]["giocatore"]
#    titolari=list(squadre[squadre["giocatore"]!=riserva].loc[sq]["giocatore"])
#    data.append([sq,capitano,riserva]+titolari)
#formazioni=pd.DataFrame(data,columns=["squadra","capitano","riserva","titolare 1","titolare 2"])
#formazioni=formazioni.set_index("squadra")
#formazioni["riposa"]=""
#formazioni.set_value(riposano.iloc[0]["squadra 1"],"riposa","(riposa)")

formazioni=s.get_formazioni(GIORNATA)


In [45]:
formazioni

,riposa,capitano,riserva,titolare 1,titolare 2
squadra,,,,,
Green House,,Bango Skank,Bango Skank,Joyopi,Plata
La threesome,(riposa),mina99,,,
I Bananari,,simone volponi,massimopud,Niko,simone volponi
Armata Brancaleone,,AdStr,Skorpiòs,Ella F.,AdStr
Living Stones,,Macleo,Vincenzo Iennaco,Macleo,Ljuset
Retroguardia Trash,,Andrea28,AlexComan,Eudes,Andrea28
Le TreMende,,Emy,sefora,Emy,Thea
Gli scrittori del quartierino,,Marty12,Federico72,camparino,Marty12
the 3 BrOthers,,AryaSophia,ettore70,ITG,AryaSophia


In [57]:
importlib.reload(melo)

elo_g1=elo_g_g1.copy()
elo_g1.columns=["squadra","rank 1"]

# c'è una componente casuale, una volta pubblicati vanno fissati
#accoppiamenti=melo.calcola_accoppiamenti(GIORNATA,formazioni,calendario,elo_g1)
# accoppiamenti.reset_index().as_matrix()

data=[['A', 1, 'D Euforiche', 'Kuno', 'Marcello', 'Kuno', 'queffe'],
       ['A', 1, 'Gli ultimi', 'crazycat', 'Ghigo', 'caipiroska', 'crazycat'],
       ['B', 1, 'Green House', 'Bango Skank', 'Bango Skank', 'Plata',
        'Joyopi'],
       ['B', 1, 'Gli scrittori del quartierino', 'Marty12', 'Federico72',
        'camparino', 'Marty12'],
       ['B', 2, 'I Bananari', 'simone volponi', 'massimopud',
        'simone volponi', 'Niko'],
       ['B', 2, 'the 3 BrOthers', 'AryaSophia', 'ettore70', 'ITG',
        'AryaSophia'],
       ['C', 1, 'Living Stones', 'Macleo', 'Vincenzo Iennaco', 'Ljuset',
        'Macleo'],
       ['C', 1, 'Le TreMende', 'Emy', 'sefora', 'Thea', 'Emy'],
       ['C', 2, 'Retroguardia Trash', 'Andrea28', 'AlexComan', 'Andrea28',
        'Eudes'],
       ['C', 2, 'Armata Brancaleone', 'AdStr', 'Skorpiòs', 'Ella F.',
        'AdStr']]
accoppiamenti=pd.DataFrame(data,columns=["girone","partita","squadra","capitano",
                                         "riserva","match 1",
                                         "match 2"]).set_index(["girone","partita"])
accoppiamenti



squadra        capitano  \
girone partita                                                  
A      1                          D Euforiche            Kuno   
       1                           Gli ultimi        crazycat   
B      1                          Green House     Bango Skank   
       1        Gli scrittori del quartierino         Marty12   
       2                           I Bananari  simone volponi   
       2                       the 3 BrOthers      AryaSophia   
C      1                        Living Stones          Macleo   
       1                          Le TreMende             Emy   
       2                   Retroguardia Trash        Andrea28   
       2                   Armata Brancaleone           AdStr   

                         riserva         match 1     match 2  
girone partita                                                
A      1                Marcello            Kuno      queffe  
       1                   Ghigo      caipiroska    crazycat  
B      1             Bango Skank           Plata      Joyopi  
       1              Federico72       camparino     Marty12  
       2              massimopud  simone volponi        Niko  
       2                ettore70             ITG  AryaSophia  
C      1        Vincenzo Iennaco          Ljuset      Macleo  
       1                  sefora            Thea         Emy  
       2               AlexComan        Andrea28       Eudes  
       2                Skorpiòs         Ella F.       AdStr

In [58]:
# Codice html per il post di riepilogo sugli accoppiamenti

prev_girone=""
for girone,partita in accoppiamenti.index.unique():
    if prev_girone!=girone:
        print("<p>&nbsp;</p>")
        print("<p><strong>Girone %s</strong></p>" % girone)
        prev_girone=girone
    p1=accoppiamenti.loc[girone,partita].iloc[0]
    p2=accoppiamenti.loc[girone,partita].iloc[1]
    print("<p>%d. %s - %s</p>" % (partita,p1["squadra"],p2["squadra"]) )
    print("<p style='margin-left: 40px;'>match 1: %s - %s</p>" % (p1["match 1"],p2["match 1"]))
    print("<p style='margin-left: 40px;'>match 2: %s - %s</p>" % (p1["match 2"],p2["match 2"]))
    

<p>&nbsp;</p>
<p><strong>Girone A</strong></p>
<p>1. D Euforiche - Gli ultimi</p>
<p style='margin-left: 40px;'>match 1: Kuno - caipiroska</p>
<p style='margin-left: 40px;'>match 2: queffe - crazycat</p>
<p>&nbsp;</p>
<p><strong>Girone B</strong></p>
<p>1. Green House - Gli scrittori del quartierino</p>
<p style='margin-left: 40px;'>match 1: Plata - camparino</p>
<p style='margin-left: 40px;'>match 2: Joyopi - Marty12</p>
<p>2. I Bananari - the 3 BrOthers</p>
<p style='margin-left: 40px;'>match 1: simone volponi - ITG</p>
<p style='margin-left: 40px;'>match 2: Niko - AryaSophia</p>
<p>&nbsp;</p>
<p><strong>Girone C</strong></p>
<p>1. Living Stones - Le TreMende</p>
<p style='margin-left: 40px;'>match 1: Ljuset - Thea</p>
<p style='margin-left: 40px;'>match 2: Macleo - Emy</p>
<p>2. Retroguardia Trash - Armata Brancaleone</p>
<p style='margin-left: 40px;'>match 1: Andrea28 - Ella F.</p>
<p style='margin-left: 40px;'>match 2: Eudes - AdStr</p>


## Fase 2. Creazione del tabellone per voti e controlli

In [59]:
importlib.reload(ms)
s=ms.N2017Spreadsheet()

# togliere quando c'è la versione definitiva delle formazioni
# old_data=s.get_giornata(GIORNATA)
old_data=None
s.set_giornata(GIORNATA,accoppiamenti,gironi,old_data)


In [ ]:
### Tabella ammissioni

In [ ]:
importlib.reload(ms)

s=ms.N2017Spreadsheet()
ammissioni=s.get_giornata(GIORNATA)
ammissioni=ammissioni[["base","verifiche"]]

old_cols=ammissioni.columns.tolist()
new_cols=[c[1] for c in old_cols]
new_cols

ammissioni.columns=new_cols
a=accoppiamenti.reset_index().set_index("match 1")[["girone","partita","squadra","capitano","riserva"]]
a["match"]=1
a
b=accoppiamenti.reset_index().set_index("match 2")[["girone","partita","squadra","capitano","riserva"]]
b["match"]=2

ammissioni=ammissioni.merge(a.append(b),left_index=True,right_index=True)
new_index=["girone","partita","match","squadra"]
new_columns=["index","riserva","ingresso riserva",
             "titolo","lunghezza","penalità","tag","link","link commento","commento valido"]

ammissioni=ammissioni.reset_index().set_index(new_index).sort_index()[new_columns]
new_columns=["titolare","riserva","ingresso riserva",
             "titolo","lunghezza","penalità","tag","link","link commento","commento valido"]
ammissioni.columns=new_columns

In [ ]:
def calc_ammissioni(row):
    row["sostituzione"]=""
    if row["ingresso riserva"]:
        sost="("+row["riserva"]+" da "+row["ingresso riserva"]+")"
        row["sostituzione"]=sost
    row["punti penalità"]=""
    if int(row["penalità"])==0: return row
    if abs(int(row["penalità"]))==1:
        row["punti penalità"]="1 punto di penalità"
    else:
        row["punti penalità"]="%d punti di penalità" % abs(int(row["penalità"]))
    return row

ammissioni=ammissioni.apply(calc_ammissioni,axis=1)
ammissioni

In [ ]:

#old_cols=ammissioni.columns.tolist()
#new_cols=[c[1] for c in old_cols]
#new_cols

#ammissioni.columns=new_cols

mu.to_table_html_multi(ammissioni[["titolare","titolo",
                                   "lunghezza"]])

## Fase 3. Recupero dati

In [31]:
importlib.reload(ms)

s=ms.N2017Spreadsheet()
voti=s.get_giornata(GIORNATA)


In [32]:
### Simulazione di riempimento del tabellone - da togliere quando ci sono i voti definitivi
importlib.reload(mu)

voti["base","titolo"]=voti["base","titolo"].apply(mu.random_title)
voti["base","ingresso riserva"]=voti["base","ingresso riserva"].apply(mu.random_riserva)
voti["verifiche","lunghezza"]=voti["verifiche","lunghezza"].apply(mu.random_len)
voti["verifiche","link"]=voti["verifiche","link"].apply(mu.random_verifica)
voti["verifiche","link commento"]=voti["verifiche","link commento"].apply(mu.random_verifica)
voti["verifiche","tag"]=voti["verifiche","tag"].apply(mu.random_verifica)
voti["verifiche","commento valido"]=voti["verifiche","commento valido"].apply(mu.random_verifica)

for G,altri in [ ("A",[("B",1),("B",2),("C",1),("C",2)]),
                 ("B",[("A",1),("C",1),("C",2)]),
                 ("C",[("A",1),("B",1),("B",2)]) ]:
    for sq in gironi[gironi[G]!="(riposo)"][G]:
        for gir,part in altri:
            for m in ["match 1","match 2"]:
                giocatore_1=accoppiamenti.loc[gir,part][m].iloc[0]
                giocatore_2=accoppiamenti.loc[gir,part][m].iloc[1]
                goal_1=random.randint(0,2)
                goal_2=2-goal_1
                voti.set_value(giocatore_1,(G,sq),goal_1)
                voti.set_value(giocatore_2,(G,sq),goal_2)

voti["verifiche","penalità"]=voti["verifiche","lunghezza"].apply(mu.calc_penalita_lungh)
for k in ["tag","link","link commento","commento valido"]:
    voti["verifiche","penalità"]+=voti["verifiche",k].apply(mu.calc_penalita)


In [33]:
voti["base","goal giocatore"]=voti[["A","B","C"]].sum(axis=1,level=1).sum(axis=1)+voti["verifiche","penalità"]
voti

base                       verifiche           \
               ingresso riserva                titolo lunghezza tag link   
titolare                                                                   
queffe                                  XcLPN Bm8 ZkG      4083   x    x   
Marcello            12-08 23:23             IYtZ4G6wl       800   x    x   
crazycat                           gmHM 4TNAExZlxVvjm      3770        x   
caipiroska          12-08 23:23          h BGZU tIZ G      2659   x    x   
Joyopi              12-08 23:23      XSn 3scf m8 ZP 4      2455   x    x   
Plata               12-08 23:23         5 9PX JDLNqII      3509   x    x   
camparino                           aXmGBPhGj2d1Rj4xE      1695   x    x   
Marty12                                   7bVPvx 5G D      1599   x    x   
Niko                              TkJG2 Z JCCBvn R PQ      2479   x    x   
simone volponi      12-08 23:23  5KG9o2yGqG ffMtOrMpQ      4594   x    x   
ettore70            12-08 23:23   y rxupXV IGGQWcY s6      7297   x    x   
AryaSophia          12-08 23:23      0B XsrN3 eSV Ez4      1594   x    x   
Macleo              12-08 23:23  3zGV6nLBrSN pQ3rV xY      4552        x   
Ljuset                                   igGz H S 4pa      5734   x    x   
Thea                                      z tqbye TKd      5261        x   
sefora                                 whM Vhtmpv DcD      3698   x    x   
Andrea28                              g BcUMLpt 9X xM      4004   x    x   
Eudes               12-08 23:23      VwAgipp b zGfJXn      6584   x    x   
AdStr                               zyTS iybXdyUhhZSU      1450   x    x   
Ella F.                            9d 5 g Y3h9NqjP Qq      2069   x    x   

                                                                A  \
               link commento commento valido penalità D Euforiche   
titolare                                                            
queffe                     x               x        0           0   
Marcello                                   x       -1           0   
crazycat                   x               x       -1           0   
caipiroska                 x               x        0           0   
Joyopi                     x               x        0           2   
Plata                      x               x        0           1   
camparino                  x                       -1           0   
Marty12                    x               x        0           1   
Niko                       x               x        0           0   
simone volponi             x                       -1           2   
ettore70                   x               x        0           2   
AryaSophia                 x                       -1           0   
Macleo                     x               x       -1           2   
Ljuset                     x               x        0           2   
Thea                                       x       -2           0   
sefora                     x                       -1           0   
Andrea28                   x               x        0           1   
Eudes                                      x       -1           2   
AdStr                      x               x        0           1   
Ella F.                    x               x        0           0   

                                                 B             \
               La threesome Gli ultimi Green House I Bananari   
titolare                                                        
queffe                    0          0           1          0   
Marcello                  0          0           1          0   
crazycat                  0          0           1          2   
caipiroska                0          0           1          2   
Joyopi                    2          0           0          0   
Plata                     2          1           0          0   
camparino                 0          2           0          0   
Marty12                   0          1  

## Calcolo dei risultati

### Risultati della seconda giornata

In [34]:
risultati=accoppiamenti.copy()

risultati=risultati.reset_index().set_index("squadra")
risultati["match 1 goal"]=0
risultati["match 1 riserva"]=False
risultati["match 2 goal"]=0
risultati["match 2 riserva"]=False
risultati["goal"]=0
risultati["penalità"]=0


for sq in risultati.index:
    m1=risultati.loc[sq]["match 1"]
    m2=risultati.loc[sq]["match 2"]
    goal_1=voti.loc[m1]["base","goal giocatore"]
    goal_2=voti.loc[m2]["base","goal giocatore"]
    pen_1=voti.loc[m1]["verifiche","penalità"]
    pen_2=voti.loc[m2]["verifiche","penalità"]
    risultati.set_value(sq,"match 1 goal",goal_1)
    risultati.set_value(sq,"match 2 goal",goal_2)
    risultati.set_value(sq,"goal",goal_1+goal_2)
    risultati.set_value(sq,"penalità",pen_1+pen_2)
    ris_1=bool(voti.loc[m1]["base","ingresso riserva"])
    risultati.set_value(sq,"match 1 riserva",ris_1)
    if ris_1: continue
    risultati.set_value(sq,"match 2 riserva",bool(voti.loc[m2]["base","ingresso riserva"]))
    
risultati=risultati.reset_index().set_index(["girone","partita"])
sc_dati=[]
for girone,partita in risultati.index.unique():
    squadra_1=risultati.loc[girone,partita]["squadra"].iloc[0]
    squadra_2=risultati.loc[girone,partita]["squadra"].iloc[1]
    goal_1=risultati.loc[girone,partita]["goal"].iloc[0]
    goal_2=risultati.loc[girone,partita]["goal"].iloc[1]
    pen_1=risultati.loc[girone,partita]["penalità"].iloc[0]
    pen_2=risultati.loc[girone,partita]["penalità"].iloc[1]
    m1_goal_1=risultati.loc[girone,partita]["match 1 goal"].iloc[0]
    m1_goal_2=risultati.loc[girone,partita]["match 1 goal"].iloc[1]
    m2_goal_1=risultati.loc[girone,partita]["match 2 goal"].iloc[0]
    m2_goal_2=risultati.loc[girone,partita]["match 2 goal"].iloc[1]
    sc_dati.append([girone,partita,squadra_1,squadra_2,goal_1,goal_2,
                    m1_goal_1,m1_goal_2,m2_goal_1,m2_goal_2,pen_1,pen_2])

risultati=risultati.reset_index().set_index(["girone","partita","squadra"])

schedina=pd.DataFrame(sc_dati,columns=["girone","partita","squadra 1","squadra 2",
                                       "goal squadra 1","goal squadra 2",
                                       "match 1 squadra 1","match 1 squadra 2",
                                       "match 2 squadra 1","match 2 squadra 2",
                                       "penalità squadra 1","penalità squadra 2"])

out_schedina=schedina.copy()
out_schedina["titolo"]=out_schedina["squadra 1"]+" - "+out_schedina["squadra 2"]
out_schedina["risultato"]=( out_schedina["goal squadra 1"].apply(str)
                            +" - "+out_schedina["goal squadra 2"].apply(str) )
out_schedina["match 1"]=( out_schedina["match 1 squadra 1"].apply(str)
                          +" - "+out_schedina["match 1 squadra 2"].apply(str) )
out_schedina["match 2"]=( out_schedina["match 2 squadra 1"].apply(str)
                          +" - "+out_schedina["match 2 squadra 2"].apply(str) )
out_schedina=out_schedina.set_index(["girone","partita"])

out_schedina=out_schedina[["titolo","match 1","match 2","risultato"]]


risultati

capitano           riserva  \
girone partita squadra                                                        
A      1       D Euforiche                         queffe              Kuno   
               Gli ultimi                     caipiroska              Ghigo   
B      1       Green House                    Bango Skank       Bango Skank   
               Gli scrittori del quartierino   Federico72        Federico72   
       2       I Bananari                      massimopud        massimopud   
               the 3 BrOthers                         ITG               ITG   
C      1       Living Stones                       Macleo  Vincenzo Iennaco   
               Le TreMende                         sefora               Emy   
       2       Retroguardia Trash               AlexComan         AlexComan   
               Armata Brancaleone                   AdStr          Skorpiòs   

                                                match 1         match 2  \
girone partita squadra                                                    
A      1       D Euforiche                       queffe        Marcello   
               Gli ultimi                      crazycat     caipiroska    
B      1       Green House                       Joyopi           Plata   
               Gli scrittori del quartierino  camparino         Marty12   
       2       I Bananari                          Niko  simone volponi   
               the 3 BrOthers                  ettore70      AryaSophia   
C      1       Living Stones                     Macleo          Ljuset   
               Le TreMende                         Thea          sefora   
       2       Retroguardia Trash              Andrea28           Eudes   
               Armata Brancaleone                 AdStr         Ella F.   

                                              match 1 goal  match 1 riserva  \
girone partita squadra                                                        
A      1       D Euforiche                               9            False   
               Gli ultimi                                6            False   
B      1       Green House                               7             True   
               Gli scrittori del quartierino             6            False   
       2       I Bananari                                5            False   
               the 3 BrOthers                            9             True   
C      1       Living Stones                             8             True   
               Le TreMende                               3            False   
       2       Retroguardia Trash                        9            False   
               Armata Brancaleone                        5            False   

                                              match 2 goal  match 2 riserva  \
girone partita squadra                                                        
A      1       D Euforiche                               8             True   
               Gli ultimi                                7             True   
B      1       Green House                               8            False   
               Gli scrittori del quartierino             6            False   
       2       I Bananari                                6             True   
               the 3 BrOthers                            6            False   
C      1       Living Stones                             7            False   
               Le TreMende                               6            False   
       2       Retroguardia Trash                        9             True   
               Armata Brancaleone                        4            False   

                                              goal  penalità  
girone partita squadra                                        
A      1       D Euforiche                      17        -1  
               Gli ultimi                       13        -1  
B      1       Green House                      1

In [35]:
schedina

,girone,partita,squadra 1,squadra 2,goal squadra 1,goal squadra 2,match 1 squadra 1,match 1 squadra 2,match 2 squadra 1,match 2 squadra 2,penalità squadra 1,penalità squadra 2
0,A,1,D Euforiche,Gli ultimi,17,13,9,6,8,7,-1,-1
1,B,1,Green House,Gli scrittori del quartierino,15,12,7,6,8,6,0,-1
2,B,2,I Bananari,the 3 BrOthers,11,15,5,9,6,6,-1,-1
3,C,1,Living Stones,Le TreMende,15,9,8,3,7,6,-1,-3
4,C,2,Retroguardia Trash,Armata Brancaleone,18,9,9,5,9,4,-1,0


In [36]:
out_schedina

titolo match 1 match 2  \
girone partita                                                                
A      1                           D Euforiche - Gli ultimi   9 - 6   8 - 7   
B      1        Green House - Gli scrittori del quartierino   7 - 6   8 - 6   
       2                        I Bananari - the 3 BrOthers   5 - 9   6 - 6   
C      1                        Living Stones - Le TreMende   8 - 3   7 - 6   
       2            Retroguardia Trash - Armata Brancaleone   9 - 5   9 - 4   

               risultato  
girone partita            
A      1         17 - 13  
B      1         15 - 12  
       2         11 - 15  
C      1          15 - 9  
       2          18 - 9

In [37]:
# output per il post dei risultati

mu.to_table_html_multi(out_schedina)


<table style='margin:auto;' border='1' width='90%'>
<th>girone</th>
<th>partita</th>
<th>titolo</th>
<th>match 1</th>
<th>match 2</th>
<th>risultato</th>
</tr>
<tr>
<th rowspan='1'>A</th>
<th rowspan='2'>1</th>
<td style='text-align:center;'>D Euforiche - Gli ultimi</td>
<td style='text-align:center;'>9 - 6</td>
<td style='text-align:center;'>8 - 7</td>
<td style='text-align:center;'>17 - 13</td>
</tr>
<tr>
<th rowspan='2'>B</th>
<td style='text-align:center;'>Green House - Gli scrittori del quartierino</td>
<td style='text-align:center;'>7 - 6</td>
<td style='text-align:center;'>8 - 6</td>
<td style='text-align:center;'>15 - 12</td>
</tr>
<tr>
<th rowspan='1'>2</th>
<td style='text-align:center;'>I Bananari - the 3 BrOthers</td>
<td style='text-align:center;'>5 - 9</td>
<td style='text-align:center;'>6 - 6</td>
<td style='text-align:center;'>11 - 15</td>
</tr>
<tr>
<th rowspan='2'>C</th>
<th rowspan='1'>1</th>
<td style='text-align:center;'>Living Stones - Le TreMende</td>
<td style='

### Classifica


Ordine dei criteri:
* classifiche avulse (scontri diretti);
* differenza reti;
* numero di reti segnate;
* numero di vittorie;
* differenza reti nell'ultimo incontro;
* numero di reti segnate nell'ultimo incontro;
* differenza reti al netto di goal assegnati o tolti dall'arbitro;
* numero di reti segnate al netto di goal assegnati o tolti dall'arbitro.

Nella prima giornata non ci sono classifiche avulse e l'unico incontro è anche l'ultimo, quindi si considerano solo:
* differenza reti;
* numero di reti segnate;
* numero di vittorie;
* differenza reti al netto di goal assegnati o tolti dall'arbitro;
* numero di reti segnate al netto di goal assegnati o tolti dall'arbitro.


In [49]:
def calc_vittorie(row):
    row["vittorie"]=0
    row["sconfitte"]=0
    row["pareggi"]=0
    if row["goal fatti"]>row["goal subiti"]:
        row["vittorie"]=1
    elif row["goal fatti"]<row["goal subiti"]:
        row["sconfitte"]=1
    else:
        row["pareggi"]=1
    #row["punti"]=3*row["vittorie"]+row["pareggi"]
    return row

q=schedina.copy()

a=q[["girone","squadra 1","goal squadra 1","goal squadra 2",
     "penalità squadra 1","penalità squadra 2"]].copy()
b=q[["girone","squadra 2","goal squadra 2","goal squadra 1",
     "penalità squadra 2","penalità squadra 1"]].copy()

a.columns=["girone","squadra","goal fatti","goal subiti","penalità fatte","penalità subite"]
b.columns=["girone","squadra","goal fatti","goal subiti","penalità fatte","penalità subite"]

classifica_g2=a.append(b).set_index(["girone","squadra"]).sort_index()
classifica_g2=classifica_g2.apply(calc_vittorie,axis=1)

df=pd.DataFrame([["A",riposano.iloc[0]["squadra 1"],0]],
                columns=["girone","squadra","goal fatti"])
df=df.set_index(["girone","squadra"])
classifica_g2=classifica_g2.append(df).fillna(0).sort_index()

base_fields=["goal fatti","goal subiti","penalità fatte",
             "penalità subite","vittorie","sconfitte","pareggi"]

classifica_g2.merge(classifica_g1[base_fields],left_index=True,right_index=True,
                    how="outer",suffixes=[" g1"," g2"]).fillna(0)

classifica=classifica_g2+classifica_g1[base_fields]

In [51]:
classifica["goal netti fatti"]=classifica["goal fatti"]-classifica["penalità fatte"]
classifica["goal netti subiti"]=classifica["goal subiti"]-classifica["penalità subite"]

classifica["differenza reti"]=classifica["goal fatti"]-classifica["goal subiti"]
classifica["differenza reti netta"]=classifica["goal netti fatti"]-classifica["goal netti subiti"]

classifica["punti"]=3*classifica["vittorie"]+classifica["pareggi"]

#df=pd.DataFrame([["A","Gli ultimi",0]],columns=["girone","squadra","goal fatti"])
#df=df.set_index(["girone","squadra"])
#classifica=classifica.append(df).fillna(0).sort_index()
classifica=classifica[["punti","vittorie","pareggi","sconfitte","differenza reti",
                       "goal fatti","goal subiti",
                       "differenza reti netta",
                       "goal netti fatti","goal netti subiti","penalità fatte","penalità subite"]]
classifica=classifica.astype(int)
classifica=classifica.sort_values(by=["punti","differenza reti","goal fatti","vittorie",
                                  "differenza reti netta","goal netti fatti"],
                                  ascending=False).sort_index(level="girone",
                                                              kind="mergesort",
                                                              sort_remaining=False)
classifica

punti  vittorie  pareggi  sconfitte  \
girone squadra                                                              
A      D Euforiche                        6         2        0          0   
       Gli ultimi                         0         0        0          1   
       La threesome                       0         0        0          1   
B      Green House                        4         1        1          0   
       Gli scrittori del quartierino      3         1        0          1   
       the 3 BrOthers                     3         1        0          1   
       I Bananari                         1         0        1          1   
C      Living Stones                      6         2        0          0   
       Retroguardia Trash                 3         1        0          1   
       Le TreMende                        3         1        0          1   
       Armata Brancaleone                 0         0        0          2   

                                      differenza reti  goal fatti  \
girone squadra                                                      
A      D Euforiche                                 17          39   
       Gli ultimi                                  -4          13   
       La threesome                               -13           9   
B      Green House                                  3          28   
       Gli scrittori del quartierino                5          29   
       the 3 BrOthers                              -4          24   
       I Bananari                                  -4          24   
C      Living Stones                               10          30   
       Retroguardia Trash                           5          29   
       Le TreMende                                 -1          25   
       Armata Brancaleone                         -14          20   

                                      goal subiti  differenza reti netta  \
girone squadra                                                             
A      D Euforiche                             22                     16   
       Gli ultimi                              17                     -4   
       La threesome                            22                    -12   
B      Green House                             25                      2   
       Gli scrittori del quartierino           24                      6   
       the 3 BrOthers                          28                     -4   
       I Bananari                              28                     -4   
C      Living Stones                           20                      6   
       Retroguardia Trash                      24                      8   
       Le TreMende                             26                      0   
       Armata Brancaleone                      34                    -14   

                                      goal netti fatti  goal netti subiti  \
girone squadra                                                              
A      D Euforiche                                  40                 24   
       Gli ultimi                                   14                 18   
       La threesome                                 10                 22   
B      Green House                                  29                 27   
       Gli scrittori del quartierino                31                 25   
       the 3 BrOthers                               26                 30   
       I Bananari                                   26                 30   
C      Living Stones                                31                 25   
       Retroguardia Trash                           32                 24   
       Le TreMende                                  28                 28   
       Armata Brancaleone                           21                 35   

                                      penalità fatte  penalità subite  
girone squadra                                                         
A  

In [52]:
## Output per il post della classifica

mu.to_table_html_multi(classifica)

<table style='margin:auto;' border='1' width='90%'>
<th>girone</th>
<th>squadra</th>
<th>punti</th>
<th>vittorie</th>
<th>pareggi</th>
<th>sconfitte</th>
<th>differenza reti</th>
<th>goal fatti</th>
<th>goal subiti</th>
<th>differenza reti netta</th>
<th>goal netti fatti</th>
<th>goal netti subiti</th>
<th>penalità fatte</th>
<th>penalità subite</th>
</tr>
<tr>
<th rowspan='3'>A</th>
<th rowspan='1'>D Euforiche</th>
<td style='text-align:center;'>6</td>
<td style='text-align:center;'>2</td>
<td style='text-align:center;'>0</td>
<td style='text-align:center;'>0</td>
<td style='text-align:center;'>17</td>
<td style='text-align:center;'>39</td>
<td style='text-align:center;'>22</td>
<td style='text-align:center;'>16</td>
<td style='text-align:center;'>40</td>
<td style='text-align:center;'>24</td>
<td style='text-align:center;'>-1</td>
<td style='text-align:center;'>-2</td>
</tr>
<tr>
<th rowspan='1'>Gli ultimi</th>
<td style='text-align:center;'>0</td>
<td style='text-align:center;'>0</t

### Nuovi punteggi Elo

In [72]:
q=risultati.copy()

def calc_giocatore(row,match):
    riserva=row["riserva"]
    giocatore=row["match %d" % match]
    sostituzione=row["match %d riserva" % match]
    if sostituzione: return riserva
    return giocatore

def calc_giocatore_1(row): return calc_giocatore(row,1)
def calc_giocatore_2(row): return calc_giocatore(row,2)

q["giocatore 1"]=q.apply(calc_giocatore_1,axis=1)
q["giocatore 2"]=q.apply(calc_giocatore_2,axis=1)

a=q[["giocatore 1","match 1 goal"]].copy()
a.columns=["giocatore","goal"]
a["match"]="1"

b=q[["giocatore 2","match 2 goal"]].copy()
b.columns=["giocatore","goal"]
b["match"]="2"

q=a.append(b).reset_index().set_index(["girone","partita","match"])
q=q.merge(q,how="outer",left_index=True,right_index=True,suffixes=[" 1"," 2"])
q=q[q["giocatore 1"]!=q["giocatore 2"]]

elo=q[["squadra 1","giocatore 1","giocatore 2","goal 1","goal 2"]]
elo.columns=["squadra","giocatore","avversario","goal fatti","goal subiti"]
elo=elo.reset_index().set_index(["girone","squadra","giocatore"]).sort_index()
elo=elo[["avversario","goal fatti","goal subiti"]].copy()
#elo["elo precedente"]=1500
#elo["elo avversario"]=1500

base_elo=elo_g_g1[["elo"]].copy()

base_elo.columns=["elo precedente"]
elo=elo.reset_index().set_index("giocatore").merge(base_elo,left_index=True,right_index=True)
base_elo.columns=["elo avversario"]
elo=elo.reset_index().set_index("avversario").merge(base_elo,left_index=True,right_index=True)
elo=elo.reset_index().set_index(["girone","squadra","giocatore"])
elo.columns=["avversario","goal fatti","goal subiti","elo precedente","elo avversario"]
elo

avversario  \
girone squadra                       giocatore                            
A      D Euforiche                   Kuno                         Ghigo   
                                     queffe                    crazycat   
       Gli ultimi                    Ghigo                         Kuno   
                                     crazycat                    queffe   
B      Gli scrittori del quartierino Marty12                      Plata   
                                     camparino              Bango Skank   
       Green House                   Bango Skank              camparino   
                                     Plata                      Marty12   
       I Bananari                    Niko                           ITG   
                                     massimopud              AryaSophia   
       the 3 BrOthers                AryaSophia              massimopud   
                                     ITG                           Niko   
C      Armata Brancaleone            AdStr                     Andrea28   
                                     Ella F.                  AlexComan   
       Le TreMende                   Thea              Vincenzo Iennaco   
                                     sefora                      Ljuset   
       Living Stones                 Ljuset                      sefora   
                                     Vincenzo Iennaco              Thea   
       Retroguardia Trash            AlexComan                  Ella F.   
                                     Andrea28                     AdStr   

                                                       goal fatti  \
girone squadra                       giocatore                      
A      D Euforiche                   Kuno                       8   
                                     queffe                     9   
       Gli ultimi                    Ghigo                      7   
                                     crazycat                   6   
B      Gli scrittori del quartierino Marty12                    6   
                                     camparino                  6   
       Green House                   Bango Skank                7   
                                     Plata                      8   
       I Bananari                    Niko                       5   
                                     massimopud                 6   
       the 3 BrOthers                AryaSophia                 6   
                                     ITG                        9   
C      Armata Brancaleone            AdStr                      5   
                                     Ella F.                    4   
       Le TreMende                   Thea                       3   
                                     sefora                     6   
       Living Stones                 Ljuset                     7   
                                     Vincenzo Iennaco           8   
       Retroguardia Trash            AlexComan                  9   
                                     Andrea28                   9   

                                                       goal subiti  \
girone squadra                       giocatore                       
A      D Euforiche                   Kuno                        7   
                                     queffe                      6   
       Gli ultimi                    Ghigo                       8   
                                     crazycat                    9   
B      Gli scrittori del quartierino Marty12                     8   
                                     camparino                   7   
       Green House                   Bango Skank                 6   
                                     Plata                       6   
       I Bananari                    Niko                        9   
                                     massimopud                  6   
       the 3 BrOthers                AryaSophia    

In [73]:
def calcola_punteggi(row):
    exp_a,exp_b=melo.punteggi_attesi(row["elo precedente"],row["elo avversario"])
    score_a,score_b=melo.punteggi_reali(row["goal fatti"],row["goal subiti"])
    row["punteggio atteso"]=exp_a
    row["punteggio reale"]=score_a
    return row

elo=elo.apply(calcola_punteggi,axis=1)

elo["scostamento"]=elo["punteggio reale"]-elo["punteggio atteso"]
mean=elo["scostamento"].mean()
sigma=elo["scostamento"].std()

for alpha in np.linspace(0,6,601):
    if mlab.normpdf(alpha*sigma,mean,sigma)<=0.0001:
        break

#x = np.linspace(mean - alpha*sigma, mean + alpha*sigma, 100)
#plt.plot(x,mlab.normpdf(x, mean, sigma))
#plt.show()

max_scostamento=alpha*sigma
max_variazione=50
K=max_variazione/max_scostamento

def norm_elo(x):
    e=round(max(1450,x))
    return int(e)
    
elo["elo nuovo"]=elo["elo precedente"]+K*elo["scostamento"]
elo["elo nuovo"]=elo["elo nuovo"].apply(norm_elo)
elo=elo.reset_index().set_index(["giocatore"]).sort_values("elo nuovo",ascending=False)
elo

,girone,squadra,avversario,goal fatti,goal subiti,elo precedente,elo avversario,punteggio atteso,punteggio reale,scostamento,elo nuovo
giocatore,,,,,,,,,,,
Vincenzo Iennaco,C,Living Stones,Thea,8,3,1517,1516,0.501439,0.727273,0.225834,1536
queffe,A,D Euforiche,crazycat,9,6,1518,1500,0.525881,0.600000,0.074119,1524
Andrea28,C,Retroguardia Trash,AdStr,9,5,1510,1504,0.508634,0.642857,0.134223,1521
Plata,B,Green House,Marty12,8,6,1500,1507,0.489928,0.571429,0.081501,1507
ITG,B,the 3 BrOthers,Niko,9,5,1493,1502,0.487051,0.642857,0.155806,1506
camparino,B,Gli scrittori del quartierino,Bango Skank,6,7,1511,1498,0.518700,0.461538,-0.057161,1506
Kuno,A,D Euforiche,Ghigo,8,7,1500,1500,0.500000,0.533333,0.033333,1503
Bango Skank,B,Green House,camparino,7,6,1498,1511,0.481300,0.538462,0.057161,1503
AlexComan,C,Retroguardia Trash,Ella F.,9,4,1483,1500,0.475555,0.692308,0.216753,1501


In [74]:
a=gironi[gironi["A"]!="(riposo)"][["A"]].copy()
a.columns=["squadra"]
a["girone"]="A"
a

b=gironi[gironi["B"]!="(riposo)"][["B"]].copy()
b.columns=["squadra"]
b["girone"]="B"
b

c=gironi[gironi["C"]!="(riposo)"][["C"]].copy()
c.columns=["squadra"]
c["girone"]="C"
c

q=a.append(b).append(c).set_index("squadra")
q=q.merge(squadre,left_index=True,right_index=True,how="outer")
q=q.reset_index().set_index("giocatore")
q

elo_g=elo.merge(q,how="outer",
                left_index=True,right_index=True)
elo_g=elo_g[["squadra_y","elo nuovo"]].fillna(1500).sort_values("elo nuovo",
                                                                           ascending=False)
elo_g["elo nuovo"]=pd.to_numeric(elo_g["elo nuovo"],downcast="signed")
elo_g.columns=["squadra","elo"]
elo_g

,squadra,elo
giocatore,,
Vincenzo Iennaco,Living Stones,1536
queffe,D Euforiche,1524
Andrea28,Retroguardia Trash,1521
Plata,Green House,1507
camparino,Gli scrittori del quartierino,1506
ITG,the 3 BrOthers,1506
Bango Skank,Green House,1503
Kuno,D Euforiche,1503
AlexComan,Retroguardia Trash,1501


In [23]:
mu.to_table_html(elo_g,with_index=True)

<table style='margin:auto;' border='1' width='90%'>
<tr><th>&nbsp;</th>
<th>squadra</th>
<th>elo</th>
</tr>
<tr>
<th>queffe</th>
<td style='text-align:center;'>D Euforiche</td>
<td style='text-align:center;'>1518</td>
</tr>
<tr>
<th>Vincenzo Iennaco</th>
<td style='text-align:center;'>Living Stones</td>
<td style='text-align:center;'>1517</td>
</tr>
<tr>
<th>Thea</th>
<td style='text-align:center;'>Le TreMende</td>
<td style='text-align:center;'>1516</td>
</tr>
<tr>
<th>camparino</th>
<td style='text-align:center;'>Gli scrittori del quartierino</td>
<td style='text-align:center;'>1511</td>
</tr>
<tr>
<th>Andrea28</th>
<td style='text-align:center;'>Retroguardia Trash</td>
<td style='text-align:center;'>1510</td>
</tr>
<tr>
<th>Marcello</th>
<td style='text-align:center;'>D Euforiche</td>
<td style='text-align:center;'>1507</td>
</tr>
<tr>
<th>Marty12</th>
<td style='text-align:center;'>Gli scrittori del quartierino</td>
<td style='text-align:center;'>1507</td>
</tr>
<tr>
<th>AdStr</th>

## Salvataggio dei risultati

In [24]:
## CSV
label="giornata_%d" % GIORNATA

# formazioni
formazioni_csv="data/formazioni_"+label+".csv"
formazioni.to_csv(formazioni_csv,index_label="squadra")

# accoppiamenti
accoppiamenti_csv="data/accoppiamenti_"+label+".csv"
accoppiamenti.to_csv(accoppiamenti_csv)

# voti
voti_csv="data/voti_"+label+".csv"
voti.to_csv(voti_csv)

# risultati
risultati_csv="data/risultati_"+label+".csv"
risultati.to_csv(risultati_csv)

# schedina
schedina_csv="data/schedina_"+label+".csv"
schedina.to_csv(schedina_csv,index=False)

# classifica
classifica_csv="data/classifica_"+label+".csv"
classifica.to_csv(classifica_csv)

# elo
elo_csv="data/elo_"+label+".csv"
elo.to_csv(elo_csv)

# elo_g
elo_g_csv="data/elo_g_"+label+".csv"
elo_g.to_csv(elo_g_csv)

# ammissioni
ammissioni_csv="data/ammissioni_"+label+".csv"
ammissioni.to_csv(ammissioni_csv)


In [56]:
x=accoppiamenti.reset_index()
x.as_matrix()


[['A', 1, 'D Euforiche', 'Kuno', 'Marcello', 'Kuno', 'queffe'],
 ['A', 1, 'Gli ultimi', 'crazycat', 'Ghigo', 'caipiroska', 'crazycat'],
 ['B', 1, 'Green House', 'Bango Skank', 'Bango Skank', 'Plata', 'Joyopi'],
 ['B',
  1,
  'Gli scrittori del quartierino',
  'Marty12',
  'Federico72',
  'camparino',
  'Marty12'],
 ['B',
  2,
  'I Bananari',
  'simone volponi',
  'massimopud',
  'simone volponi',
  'Niko'],
 ['B', 2, 'the 3 BrOthers', 'AryaSophia', 'ettore70', 'ITG', 'AryaSophia'],
 ['C', 1, 'Living Stones', 'Macleo', 'Vincenzo Iennaco', 'Ljuset', 'Macleo'],
 ['C', 1, 'Le TreMende', 'Emy', 'sefora', 'Thea', 'Emy'],
 ['C', 2, 'Retroguardia Trash', 'Andrea28', 'AlexComan', 'Andrea28', 'Eudes'],
 ['C', 2, 'Armata Brancaleone', 'AdStr', 'Skorpiòs', 'Ella F.', 'AdStr']]